In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
rf = joblib.load('../Models/rf_model.pkl')
scaler = joblib.load('../Models/scaler.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\devin\\AI-agriculture-yield-prediction\\Models\\rf_model.pkl'

In [ ]:
df = pd.read_csv('../Models/test_predictions.csv')

In [ ]:
r2 = r2_score(df['Actual'], df['Predicted'])
mae = mean_absolute_error(df['Actual'], df['Predicted'])
rmse = mean_squared_error(df['Actual'], df['Predicted'])
print(r2*100)
print(mae)
print(rmse)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(6,5))
sns.scatterplot(x=df['Actual'], y=df['Predicted'])
plt.xlabel("Actual Yield (kg/ha)")
plt.ylabel("Predicted Yield (kg/ha)")
plt.title("Actual vs Predicted Rice Yield")
plt.savefig('../Plots/actual_vs_predicted.png')
plt.grid(True)
plt.show()

In [ ]:
residuals = df['Actual'] - df['Predicted']
sns.histplot(residuals, kde=True)
plt.title("Residuals Distribution")
plt.xlabel("Prediction Error")
plt.savefig('../Plots/Prediction_error.png')
plt.show()

**ERROR ANALYSIS**

*To Identify patterns in misclassified/mispredicted rows*

In [7]:
df['Residual'] = df['Actual']-df['Predicted']
df['Abs_Error']= df['Residual'].abs()

In [8]:
mispredicted_rows = df.sort_values(by='Abs_Error', ascending =False)
display(mispredicted_rows)

In [9]:
df1= pd.read_csv('../Datasets/rice_data_outlier_removed.csv')
df1.head()

In [10]:
m_index= mispredicted_rows.index
error_df= df1.loc[m_index]
error_df['Actual'] = mispredicted_rows['Actual']
error_df['Predicted'] = mispredicted_rows['Predicted']
error_df['Abs_Error'] = mispredicted_rows['Abs_Error']
error_df.head()

In [11]:
error_df.shape

In [12]:
state_error = error_df.groupby(['State Name'])['Abs_Error'].mean().sort_values(ascending=False)
print("Avg Absolute error per state")
print(state_error.head())
year_error = error_df.groupby(['Year'])['Abs_Error'].mean().sort_values(ascending=False)
print("Evg Absolute Error per year")
print(year_error.head())

In [13]:
cols =  error_df[['Year','State Name' , 'RICE AREA (1000 ha)','RICE PRODUCTION (1000 tons)']]
for col in cols:
    plt.figure(figsize=(8,5))
    if error_df[col].dtype == 'object':
        sns.boxplot(x=col , y='Abs_Error' , data=error_df)
        plt.xticks(rotation= 90)
        plt.title(f'Error Distribution by {col}')
    else:
        sns.scatterplot(x=col , y='Abs_Error', data = error_df)
        plt.xticks(rotation=90)
        plt.title(f'Error Distribution by {col}')
plt.tight_layout()
plt.show()

In [14]:
state_error = error_df.groupby('State Name')['Abs_Error'].mean().sort_values(ascending=False)
year_error = error_df.groupby('Year')['Abs_Error'].mean().sort_values(ascending=False)
print("Top 10 States by prediction error:")
print(state_error.head(10))
print("\nError by Year:")
print(year_error)

In [15]:
plt.figure(figsize=(8,6))
sns.scatterplot(x='Actual', y='Predicted', data=mispredicted_rows, alpha=0.6)
plt.plot([mispredicted_rows['Actual'].min(), mispredicted_rows['Actual'].max()],
         [mispredicted_rows['Actual'].min(), mispredicted_rows['Actual'].max()],
         color='red', linestyle='--')  # ideal predictions line
plt.title("Actual vs Predicted")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()